In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/moai-2025-training/test_images.pt
/kaggle/input/moai-2025-training/train_images.pt
/kaggle/input/moai-2025-training/train_labels.csv


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# !pip install torchvision
import torchvision

import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd

# --- 小題1：數據讀取與預處理 ---
# 加载数据
images_raw = torch.load('/kaggle/input/moai-2025-training/train_images.pt')  # 假设形状是 [N, H, W]
labels_raw = pd.read_csv('/kaggle/input/moai-2025-training/train_labels.csv')
labels_raw.info()
# 检查原始数据形状
print("Raw images shape:", images_raw.shape)
print("Labels shape:", labels_raw.shape)

# 转换 labels 为 Tensor
labels = torch.tensor(labels_raw['label'].values, dtype=torch.long)  # 确保是整数类型

# 添加通道维度并转换为 float32
images = images_raw.unsqueeze(1).float()  # 形状变为 [N, 1, H, W]

# 检查处理后的形状
print("Processed images shape:", images.shape)
print("Processed labels shape:", labels.shape)

# 确保样本数量一致
assert images.shape[0] == labels.shape[0], "样本数量不匹配！"

# 归一化（应在划分数据集后进行，这里仅为演示）
images = (images - images.mean()) / images.std()

# 划分训练集和验证集
dataset = TensorDataset(images, labels)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# --- 小題2：構建CNN模型 ---
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # 添加 padding 保持尺寸
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 假设输入是 28x28，经过两次池化后是 7x7
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
# --- 小題3：訓練模型 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [15]:
for epoch in range(55):
    # 训练阶段
    model.train()
    train_loss, train_correct = 0, 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(labels).sum().item()
    
    # 验证阶段
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()

    # 打印结果
    train_loss /= len(train_dataset)
    train_acc = train_correct / len(train_dataset)
    val_loss /= len(val_dataset)
    val_acc = val_correct / len(val_dataset)
    
    print(f"Epoch {epoch+1}/5")
    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.4f}\n")

Epoch 1/5
Train Loss: 0.1934 | Acc: 0.9450
Val Loss: 0.0739 | Acc: 0.9780

Epoch 2/5
Train Loss: 0.0547 | Acc: 0.9826
Val Loss: 0.0535 | Acc: 0.9834

Epoch 3/5
Train Loss: 0.0378 | Acc: 0.9885
Val Loss: 0.0506 | Acc: 0.9848

Epoch 4/5
Train Loss: 0.0275 | Acc: 0.9911
Val Loss: 0.0468 | Acc: 0.9852

Epoch 5/5
Train Loss: 0.0221 | Acc: 0.9927
Val Loss: 0.0427 | Acc: 0.9872

Epoch 6/5
Train Loss: 0.0151 | Acc: 0.9952
Val Loss: 0.0498 | Acc: 0.9864

Epoch 7/5
Train Loss: 0.0121 | Acc: 0.9962
Val Loss: 0.0439 | Acc: 0.9898

Epoch 8/5
Train Loss: 0.0109 | Acc: 0.9965
Val Loss: 0.0523 | Acc: 0.9875

Epoch 9/5
Train Loss: 0.0083 | Acc: 0.9976
Val Loss: 0.0522 | Acc: 0.9881

Epoch 10/5
Train Loss: 0.0067 | Acc: 0.9978
Val Loss: 0.0461 | Acc: 0.9892

Epoch 11/5
Train Loss: 0.0065 | Acc: 0.9978
Val Loss: 0.0579 | Acc: 0.9878

Epoch 12/5
Train Loss: 0.0070 | Acc: 0.9977
Val Loss: 0.0552 | Acc: 0.9886

Epoch 13/5
Train Loss: 0.0033 | Acc: 0.9990
Val Loss: 0.0558 | Acc: 0.9883

Epoch 14/5
Train Loss

In [17]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

# 1. 加载测试集数据
test_images_raw = torch.load('/kaggle/input/moai-2025-training/test_images.pt')  # 假设形状是 [N, 28, 28]
test_ids = [f"{i}" for i in range(len(test_images_raw))]
 # 假设有包含id的CSV文件

# 2. 数据预处理（与训练时相同）
# 添加通道维度并转换为float32
test_images = test_images_raw.unsqueeze(1).float()  # 形状变为 [N, 1, 28, 28]

# 归一化（使用训练时的均值和标准差）
# 注意：这里应该使用训练时计算的mean和std，这里简化为使用测试集自己的统计量
test_images = (test_images - test_images.mean()) / test_images.std()

# 3. 创建测试集DataLoader
test_dataset = TensorDataset(test_images)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 4. 模型预测
model.eval()  # 设置为评估模式
all_preds = []

with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs[0].to(device)  # 因为TensorDataset返回的是元组
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

# 5. 创建结果DataFrame
results = pd.DataFrame({
    'id': test_ids,
    'label': all_preds
})

# 6. 保存为CSV文件
results.to_csv('/kaggle/working/submission.csv', index=False)

print("预测结果已保存为 submission.csv")
print("前5行结果预览:")
print(results.head())

/tmp/ipykernel_31/1010977431.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_images_raw = torch.load('/kaggle/input/moai-2025-training/test_images.pt')  # 假设形状是 [N,

预测结果已保存为 submission.csv
前5行结果预览:
  id  label
0  0      7
1  1      2
2  2      1
3  3      0
4  4      4
